In [444]:
#import tensorflow as tf
#from tensorflow.keras import datasets, layers, models
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [514]:
BASE_DIR = "C:/Araf/Machine_Learning/Course-3/Images/project_images/images"
OUTPUT_BASE_DIR="C:/Araf/Machine_Learning/Course-3/Images/project_images/output"
OUTPUT_IMAGE_DIR_DISEASED="C:/Araf/Machine_Learning/Course-3/Images/project_images/output/subset/diseased/"
OUTPUT_IMAGE_DIR_HEALTHY="C:/Araf/Machine_Learning/Course-3/Images/project_images/output/subset/healthy/"
fraction_val1=0.3
fraction_val2=0.3
#"C:/Araf/Machine_Learning/Course-3/Images/project_images/output/Leaf_subset1.csv"
OUTPUT_FILE1_NAME="leaf_subset1.csv"
OUTPUT_FILE2_NAME="leaf_subset2.csv"

In [515]:
BASE_DIR

'C:/Araf/Machine_Learning/Course-3/Images/project_images/images'

In [516]:
image_paths = []
for plant in os.listdir(BASE_DIR):
    for plant_type in os.listdir(f"{BASE_DIR}/{plant}"):
        for plant_image in os.listdir(f"{BASE_DIR}/{plant}/{plant_type}"):
            image_paths.append(f"{BASE_DIR}/{plant}/{plant_type}/{plant_image}")
            
print("loading paths done, loaded", len(image_paths))

loading paths done, loaded 4236


In [517]:
datastr=""
tokens=[]
dataList=[]

for i in image_paths:
    tokens=i.split("/")
    tokenslen=len(tokens)
    #print(i, "belongs to:", tokens[7], " and is of type :", tokens[8])
    subdataList=[]
    subdataList.append(i)
    subdataList.append(tokens[(tokenslen-1)])
    subdataList.append(tokens[(tokenslen-3)])
    subdataList.append(tokens[(tokenslen-2)])
    #datastr="['"+i+"','"+tokens[7]+"','"+tokens[8]+"']"
    dataList.append(subdataList)

In [518]:
dfLeaf = pd.DataFrame(dataList, columns=['filepath', 'filename','plant_type', 'leaf_label'])
#dfLeaf.index.rename('slno')

In [519]:
dfLeaf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4236 entries, 0 to 4235
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   filepath    4236 non-null   object
 1   filename    4236 non-null   object
 2   plant_type  4236 non-null   object
 3   leaf_label  4236 non-null   object
dtypes: object(4)
memory usage: 132.5+ KB


In [520]:
print("original Leaf df size: ", dfLeaf.shape[0])

original Leaf df size:  4236


In [521]:
dfgroup=dfLeaf.groupby(["plant_type", "leaf_label"]).size().reset_index(name='leaf_count_by_type_and_Label')

In [522]:
dfgroup['leaf_count_by_type_and_Label_fraction']=dfgroup[['leaf_count_by_type_and_Label']]*fraction_val2
dfgroup['leaf_count_by_type_and_Label_fraction']=dfgroup['leaf_count_by_type_and_Label_fraction'].astype(int)

In [523]:
dfgroup.head(2)

,plant_type,leaf_label,leaf_count_by_type_and_Label,leaf_count_by_type_and_Label_fraction
0,Alstonia_Scholaris_P2,diseased,254,76
1,Alstonia_Scholaris_P2,healthy,179,53


In [524]:
dfgroup1=dfLeaf.groupby(["plant_type"]).size().reset_index(name='leaf_counts_by_type')

In [525]:
dfgroup1['leaf_counts_by_type_fraction']=dfgroup1[['leaf_counts_by_type']]*fraction_val1

In [531]:
dfgroup1['leaf_counts_by_type_fraction']=dfgroup1['leaf_counts_by_type_fraction'].astype(int)

In [532]:
dfgroup3=pd.merge(dfgroup1, dfgroup, on='plant_type')

In [533]:
dfgroup3.head()

,plant_type,leaf_counts_by_type,leaf_counts_by_type_fraction,leaf_label,leaf_count_by_type_and_Label,leaf_count_by_type_and_Label_fraction
0,Alstonia_Scholaris_P2,433,129,diseased,254,76
1,Alstonia_Scholaris_P2,433,129,healthy,179,53
2,Arjun_P1,452,135,diseased,232,69
3,Arjun_P1,452,135,healthy,220,66
4,Chinar_P11,223,66,diseased,120,36


In [534]:
print("Total # of leafs: ", dfgroup3[['leaf_counts_by_type']].sum()/2, "****", (dfgroup3[['leaf_counts_by_type_fraction']].sum())/2, " ", dfgroup3[['leaf_count_by_type_and_Label_fraction']].sum())

Total # of leafs:  leaf_counts_by_type    4236.0
dtype: float64 **** leaf_counts_by_type_fraction    1265.0
dtype: float64   leaf_count_by_type_and_Label_fraction    1262
dtype: int64


In [536]:
for index, row in dfgroup3.iterrows():
    v_leaf_counts_by_type_fraction=row['leaf_counts_by_type_fraction']
    v_subsetcount=row['leaf_count_by_type_and_Label']
    if (v_leaf_counts_by_type_fraction>v_subsetcount):
        print( "we need to do some adjustment for ", row)

In [537]:
print("total# of leafs: ",dfgroup3[['leaf_counts_by_type']].sum(), " and subset of leaf is ", dfgroup3[['leaf_count_by_type_and_Label_fraction']].sum())

total# of leafs:  leaf_counts_by_type    8472
dtype: int64  and subset of leaf is  leaf_count_by_type_and_Label_fraction    1262
dtype: int64


In [538]:
#dfgroup

In [539]:
#Creating a subset of data by taking a fraction data based on leaf counts by Plant type and leaf label

In [463]:
image_subset_paths = []
imageList1=[]

print("Create a leaf data frame by taking a subset of records based on leaf_counts_by_type_fraction variable")
totalleafcount=dfgroup3[['leaf_count_by_type_and_Label_fraction']].sum()
print("Total # of leafs to be used in this dataset is: ",totalleafcount)
for index, row in dfgroup3.iterrows():
    #print("For the plant: ", row['plant_type'], " and Leaf Type as: ", row['leaf_label'], " the #sample leafs to be taken is: ", row['subsetcount'] )
    plant_type=row['plant_type']
    leaf_label=row['leaf_label']
    sample_leaf_count=row['leaf_count_by_type_and_Label_fraction'] 
    leaf_image_list=os.listdir(f"{BASE_DIR}/{plant_type}/{leaf_label}")
    i=0
    print("Creating a subdataset of size" , sample_leaf_count, " leaf images for ", plant_type, " and leaf type ", leaf_label, ":")
    while  i < sample_leaf_count:
        subimageList=[]
        #image_subset_paths.append(f"{BASE_DIR}/{plant_type}/{leaf_label}/{leaf_image_list}")
        image_path=f"{BASE_DIR}/{plant_type}/{leaf_label}/" + leaf_image_list[i]
        subimageList.append(leaf_image_list[i])
        subimageList.append(image_path)
        subimageList.append(leaf_label)
        imageList1.append(subimageList)
        #print("\tfor the image name " ,leaf_image_list[i], ", the file path is ", image_path, "and the Label is ", leaf_label)
        i=i+1
    

Create a leaf data frame by taking a subset of records based on leaf_counts_by_type_fraction variable
Total # of leafs to be used in this dataset is:  leaf_count_by_type_and_Label_fraction    415
dtype: int64
Creating a subdataset of size 25  leaf images for  Alstonia_Scholaris_P2  and leaf type  diseased :
Creating a subdataset of size 17  leaf images for  Alstonia_Scholaris_P2  and leaf type  healthy :
Creating a subdataset of size 23  leaf images for  Arjun_P1  and leaf type  diseased :
Creating a subdataset of size 22  leaf images for  Arjun_P1  and leaf type  healthy :
Creating a subdataset of size 12  leaf images for  Chinar_P11  and leaf type  diseased :
Creating a subdataset of size 10  leaf images for  Chinar_P11  and leaf type  healthy :
Creating a subdataset of size 14  leaf images for  Gauva_P3  and leaf type  diseased :
Creating a subdataset of size 27  leaf images for  Gauva_P3  and leaf type  healthy :
Creating a subdataset of size 34  leaf images for  Jamun_P5  and leaf

In [540]:
dfimagesubset=pd.DataFrame(imageList1, columns=['image_name','image_file_path', 'label'])

In [542]:
csv_filepath=OUTPUT_BASE_DIR+"/"+OUTPUT_FILE1_NAME
dfimagesubset.to_csv(csv_filepath, index_label ='slno')

In [543]:
df11=dfgroup3[['plant_type', 'leaf_counts_by_type_fraction']]

In [544]:
df11

,plant_type,leaf_counts_by_type_fraction
0,Alstonia_Scholaris_P2,129
1,Alstonia_Scholaris_P2,129
2,Arjun_P1,135
3,Arjun_P1,135
4,Chinar_P11,66
5,Chinar_P11,66
6,Gauva_P3,125
7,Gauva_P3,125
8,Jamun_P5,187
9,Jamun_P5,187


In [545]:
df12=df11.drop_duplicates(subset = "plant_type").reset_index()

In [546]:
df12.head(1)

,index,plant_type,leaf_counts_by_type_fraction
0,0,Alstonia_Scholaris_P2,129


In [547]:
#Creating a subset of data by taking a fraction data based on leaf counts by Plant type 

In [548]:
imageList2=[]
for index, row in df12.iterrows():
    cntr=row['leaf_counts_by_type_fraction']/2
    cntr=int(cntr)
    plant_type=row['plant_type']
    #leaf_label=row['leaf_label']
    leaf_image_list1=os.listdir(f"{BASE_DIR}/{plant_type}/diseased")
    i=0
    j=0
    #print(" leaf images for ", plant_type, " and leaf type as diseased")
    while  i < cntr:
        subimageList=[]
        #image_subset_paths.append(f"{BASE_DIR}/{plant_type}/{leaf_label}/{leaf_image_list}")
        image_path=f"{BASE_DIR}/{plant_type}/diseased/" + leaf_image_list1[i]
        subimageList.append(leaf_image_list1[i])
        subimageList.append(image_path)
        subimageList.append("diseased")
        imageList2.append(subimageList)
        #print("\tfor the image name " ,leaf_image_list[i], ", the file path is ", image_path, "and the Label is diseased")
        i=i+1
    leaf_image_list2=os.listdir(f"{BASE_DIR}/{plant_type}/healthy")    
    while  j < cntr:
        subimageList=[]
        #image_subset_paths.append(f"{BASE_DIR}/{plant_type}/{leaf_label}/{leaf_image_list}")
        image_path=f"{BASE_DIR}/{plant_type}/healthy/" + leaf_image_list2[j]
        subimageList.append(leaf_image_list1[j])
        subimageList.append(image_path)
        subimageList.append("healthy")
        imageList2.append(subimageList)
        #print("\tfor the image name " ,leaf_image_list2[j], ", the file path is ", image_path, "and the Label is healthy")
        j=j+1
dfimagesubset2=pd.DataFrame(imageList2, columns=['image_name','image_file_path', 'label'])
csv_filepath=OUTPUT_BASE_DIR+"/"+OUTPUT_FILE2_NAME
dfimagesubset2.to_csv(csv_filepath, index_label ='slno')